In [1]:
pip install yfinance

In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from termcolor import colored
import requests
import io
import time
import os
import json
from bs4 import BeautifulSoup


class StockAnalyzer:
    def __init__(self):
        self.base_dir = 'stock_analysis'
        self.stock_lists_dir = os.path.join(self.base_dir, 'stock_lists')
        self.results_dir = os.path.join(self.base_dir, 'daily_results')
        self.setup_directories()

    def setup_directories(self):
        """Create necessary directories if they don't exist"""
        for directory in [self.base_dir, self.stock_lists_dir, self.results_dir]:
            if not os.path.exists(directory):
                os.makedirs(directory)

    def should_update_stock_list(self, exchange):
        """Check if stock list needs updating (weekly update)"""
        list_file = os.path.join(self.stock_lists_dir, f'{exchange}_stocks.json')
        if not os.path.exists(list_file):
            return True
        # Check if file is older than 7 days
        file_time = datetime.fromtimestamp(os.path.getmtime(list_file))
        return (datetime.now() - file_time).days >= 7

    def save_stock_list(self, symbols, exchange):
        """Save stock list to JSON file"""
        list_file = os.path.join(self.stock_lists_dir, f'{exchange}_stocks.json')
        data = {
            'symbols': symbols,
            'last_updated': datetime.now().strftime('%Y-%m-%d')
        }
        with open(list_file, 'w') as f:
            json.dump(data, f)

    def load_stock_list(self, exchange):
        """Load stock list from JSON file"""
        list_file = os.path.join(self.stock_lists_dir, f'{exchange}_stocks.json')
        with open(list_file, 'r') as f:
            data = json.load(f)
        return data['symbols']

    def download_nse_stocks(self):
        """Download NSE stock list if needed"""
        if self.should_update_stock_list('NSE'):
            try:
                url = "https://archives.nseindia.com/content/equities/EQUITY_L.csv"
                response = requests.get(url)
                df = pd.read_csv(io.StringIO(response.content.decode('utf-8')))
                symbols = df['SYMBOL'].tolist()
                self.save_stock_list(symbols, 'NSE')
                print("NSE stock list updated")
                return symbols
            except Exception as e:
                print(f"Error downloading NSE stocks: {e}")
                if os.path.exists(os.path.join(self.stock_lists_dir, 'NSE_stocks.json')):
                    return self.load_stock_list('NSE')
                return []
        return self.load_stock_list('NSE')

    def download_bse_stocks(self):
        """Download BSE stock list if needed"""
        if self.should_update_stock_list('BSE'):
            try:
                # Using the BSE list from static file or an appropriate source
                url = "https://www.bseindia.com/corporates/List_Scrips.aspx"
                response = requests.get(url)
                soup = BeautifulSoup(response.content, 'html.parser')
                table = soup.find('table', {'id': 'ContentPlaceHolder1_tblData'})
                symbols = []

                if table:
                    rows = table.find_all('tr')[1:]  # Skip header row
                    for row in rows:
                        cols = row.find_all('td')
                        if cols:
                            bse_code = cols[0].text.strip()
                            symbols.append(bse_code)

                if symbols:
                    self.save_stock_list(symbols, 'BSE')
                    print("BSE stock list updated")
                    return symbols
            except Exception as e:
                print(f"Error downloading BSE stocks: {e}")
                if os.path.exists(os.path.join(self.stock_lists_dir, 'BSE_stocks.json')):
                    return self.load_stock_list('BSE')
                return []
        return self.load_stock_list('BSE')

    def calculate_rsi(self, data, periods=14):
        """Calculate RSI"""
        delta = data.diff()
        gain = delta.where(delta > 0, 0).rolling(window=periods).mean()
        loss = (-delta.where(delta < 0, 0)).rolling(window=periods).mean()
        rs = gain / loss
        return 100 - (100 / (1 + rs))

    def get_stock_data(self, symbol, exchange='NSE', days=200):
        """Fetch stock data"""
        try:
            ticker_symbol = f"{symbol}.NS" if exchange == 'NSE' else f"{symbol}.BO"
            stock = yf.Ticker(ticker_symbol)

            end_date = datetime.now()
            start_date = end_date - timedelta(days=days)
            hist_data = stock.history(start=start_date, end=end_date)

            if hist_data.empty:
                return None, None, None

            hist_data['RSI'] = self.calculate_rsi(hist_data['Close'])
            latest_price = hist_data['Close'].iloc[-1]
            latest_rsi = hist_data['RSI'].iloc[-1]

            return latest_price, hist_data, latest_rsi
        except Exception as e:
            print(f"Error fetching data for {symbol}: {e}")
            return None, None, None

    def analyze_stocks(self, symbols, exchange, batch_size=100):    #, min_volume=50000
        """Analyze stocks in batches"""
        results = []
        total_symbols = len(symbols)
        processed = 0

        for i in range(0, total_symbols, batch_size):
            batch_symbols = symbols[i:i + batch_size]
            processed += len(batch_symbols)
            print(f"\nProcessing {exchange} batch {i // batch_size + 1}/{(total_symbols + batch_size - 1) // batch_size}")
            print(f"Progress: {processed}/{total_symbols} stocks ({(processed / total_symbols * 100):.1f}%)")

            for symbol in batch_symbols:
                latest_price, hist_data, latest_rsi = self.get_stock_data(symbol, exchange)

                if all(v is not None for v in [latest_price, hist_data, latest_rsi]):
                    volume = hist_data['Volume'].iloc[-1] if 'Volume' in hist_data else 0
                    if volume >= min_volume:
                        results.append({
                            'Symbol': symbol,
                            'Exchange': exchange,
                            'Latest_Price': latest_price,
                            'RSI': latest_rsi,
                            'Volume': volume,
                            'Date': datetime.now().strftime('%Y-%m-%d')
                        })

            time.sleep(1)

        return pd.DataFrame(results)

    def run_daily_analysis(self):
        """Run daily analysis and save results"""
        current_date = datetime.now().strftime('%Y%m%d')

        # Get stock lists (cached if available)
        print("Loading stock lists...")
        bse_symbols = self.download_bse_stocks()
        nse_symbols = self.download_nse_stocks()

        print(f"\nAnalyzing {len(nse_symbols)} NSE and {len(bse_symbols)} BSE stocks")

        # Analyze stocks
        bse_results = self.analyze_stocks(bse_symbols, 'BSE')
        nse_results = self.analyze_stocks(nse_symbols, 'NSE')

        # Save daily results
        for exchange, results in [('NSE', nse_results), ('BSE', bse_results)]:
            if not results.empty:
                # Save full results
                results.to_csv(os.path.join(
                    self.results_dir,
                    f'{exchange.lower()}_analysis_{current_date}.csv'
                ), index=False)

                # Save high RSI stocks
                high_rsi = results[results['RSI'] >= 40].sort_values('RSI', ascending=False)
                high_rsi.to_csv(os.path.join(
                    self.results_dir,
                    f'{exchange.lower()}_high_rsi_{current_date}.csv'
                ), index=False)

                # Print high RSI stocks
                print(f"\n{exchange} Stocks with RSI >= 40:")
                for _, row in high_rsi.iterrows():
                    print(colored(
                        f"{row['Symbol']}: RSI = {row['RSI']:.2f}, "
                        f"Price = ₹{row['Latest_Price']:.2f}",
                        'green', attrs=['bold']
                    ))

        print(f"\nDaily analysis complete. Results saved in {self.results_dir}")


if __name__ == "__main__":
    analyzer = StockAnalyzer()
    analyzer.run_daily_analysis()


$500002.BO: possibly delisted; no timezone found
$500003.BO: possibly delisted; no timezone found


Loading stock lists...

Analyzing 2062 NSE and 3026 BSE stocks

Processing BSE batch 1/31
Progress: 100/3026 stocks (3.3%)


$500010.BO: possibly delisted; no timezone found
$500024.BO: possibly delisted; no timezone found
$500029.BO: possibly delisted; no price data found  (1d 2024-06-22 14:05:15.091353 -> 2025-01-08 14:05:15.091353)
$500032.BO: possibly delisted; no timezone found
$500033.BO: possibly delisted; no timezone found
$500038.BO: possibly delisted; no timezone found
$500040.BO: possibly delisted; no timezone found
$500042.BO: possibly delisted; no timezone found
$500048.BO: possibly delisted; no timezone found
$500049.BO: possibly delisted; no timezone found
$500052.BO: possibly delisted; no timezone found
$500055.BO: possibly delisted; no price data found  (1d 2024-06-22 14:05:30.520130 -> 2025-01-08 14:05:30.520130)
$500074.BO: possibly delisted; no timezone found
$500078.BO: possibly delisted; no timezone found
$500084.BO: possibly delisted; no timezone found
$500086.BO: possibly delisted; no timezone found
$500087.BO: possibly delisted; no timezone found
$500092.BO: possibly delisted; no tim


Processing BSE batch 2/31
Progress: 200/3026 stocks (6.6%)


$500188.BO: possibly delisted; no timezone found
$500191.BO: possibly delisted; no timezone found
$500193.BO: possibly delisted; no timezone found
$500212.BO: possibly delisted; no timezone found
$500231.BO: possibly delisted; no timezone found
$500238.BO: possibly delisted; no timezone found
$500245.BO: possibly delisted; no timezone found
$500247.BO: possibly delisted; no timezone found
$500249.BO: possibly delisted; no timezone found
$500251.BO: possibly delisted; no timezone found
$500253.BO: possibly delisted; no timezone found
$500254.BO: possibly delisted; no price data found  (1d 2024-06-22 14:07:12.468929 -> 2025-01-08 14:07:12.468929)
$500265.BO: possibly delisted; no timezone found
$500271.BO: possibly delisted; no timezone found
$500280.BO: possibly delisted; no timezone found
$500284.BO: possibly delisted; no timezone found
$500288.BO: possibly delisted; no timezone found
$500290.BO: possibly delisted; no timezone found
$500300.BO: possibly delisted; no timezone found
$500

...


Processing NSE batch 10/21
Progress: 1000/2062 stocks (48.5%)


$KALYANI.NS: possibly delisted; no price data found  (1d 2024-06-22 14:46:45.617575 -> 2025-01-08 14:46:45.617575)



Processing NSE batch 11/21
Progress: 1100/2062 stocks (53.3%)

Processing NSE batch 12/21
Progress: 1200/2062 stocks (58.2%)


$MOKSH-RE.NS: possibly delisted; no price data found  (1d 2024-06-22 14:47:24.085459 -> 2025-01-08 14:47:24.085459)



Processing NSE batch 13/21
Progress: 1300/2062 stocks (63.0%)

Processing NSE batch 14/21
Progress: 1400/2062 stocks (67.9%)

Processing NSE batch 15/21
Progress: 1500/2062 stocks (72.7%)

Processing NSE batch 16/21
Progress: 1600/2062 stocks (77.6%)

Processing NSE batch 17/21
Progress: 1700/2062 stocks (82.4%)

Processing NSE batch 18/21
Progress: 1800/2062 stocks (87.3%)

Processing NSE batch 19/21
Progress: 1900/2062 stocks (92.1%)

Processing NSE batch 20/21
Progress: 2000/2062 stocks (97.0%)

Processing NSE batch 21/21
Progress: 2062/2062 stocks (100.0%)

NSE Stocks with RSI >= 40:
BGRENERGY: RSI = 100.00, Price = ₹114.39
FSC: RSI = 96.77, Price = ₹2.49
PTCIL: RSI = 92.66, Price = ₹16925.00
INDOTHAI: RSI = 91.85, Price = ₹1570.45
AVONMORE: RSI = 89.50, Price = ₹27.32
LUPIN: RSI = 87.71, Price = ₹2357.65
AXISCADES: RSI = 87.66, Price = ₹761.00
PGIL: RSI = 87.29, Price = ₹1571.95
INDIACEM: RSI = 84.63, Price = ₹377.60
GLENMARK: RSI = 83.89, Price = ₹1642.05
APARINDS: RSI = 83.30, 